In [1]:
import pyspark

In [2]:
sc

In [24]:
import pandas 
from math import radians, cos, sin, asin, sqrt
import time
import datetime
import pyspark
from operator import add
import geopandas as gpd
from shapely.geometry import Point
import sys                                                                

ct_shp = gpd.read_file(path_ct)
crs = {'init': 'epsg:2263'}
ct_shp.to_crs(crs)

# refer column names & loc in /output/complaints_column.csv
def ft_header(row):
    return row.split(',')[0].isdigit()

def ft_noise(row):
	'''
	this function filters complaint type with Noise
	'''
	return row[4][:5].lower() == 'noise'

def mp_col(row):
    '''
    mapping info that we need
    Unique, Created Date, Closed Date, Agency, Complaint Type, Latitude, Longitude, X Coordinate (State Plane),\
    Y Coordinate (State Plane), Incident Zip
    '''
    import datetime
    row = row.split(',')
    
    try:
    # convert to datetime
        row[1] = datetime.datetime.strptime(row[1], '%m/%d/%Y %I:%M:%S %p')
        row[2] = datetime.datetime.strptime(row[2], '%m/%d/%Y %I:%M:%S %p')
    except ValueError:
        row = row
    return (row[0], row[1], row[2], row[3], row[5], row[49], row[50], row[24], row[25], row[8])

def ft_havetime(row):
    return type(row[1]) == datetime.datetime

def ft_have_geo(row):
    if '.' in row[8]:
        return (row[8].split('.')[1].isdigit()) and (row[8].split('.')[0].isdigit())
    else:
        return (row[8].isdigit()) and (row[7].isdigit())
    
def mp_groupbykey(row):
    '''
    set key
    '''
    row, ct = row
    time = row[1]
    zipcode = row[9]
    year = time.year
    month = time.month
    day = time.day
    hour = time.hour
    minute = time.minute
    #return ((year, month, day, hour, zipcode), 1)
    return ((year, ct), 1)

def mp_sjoin_ct(row):
    #load in ct file

    X = float(row[7])
    Y = float(row[8])
    geo_point = Point((X,Y))
    ct_number =  ct_shp[ct_shp.geometry.contains(geo_point)].iloc[0,0]
    return row, ct_number

def ft_try(row):
    weird = row[7]
    return row[7] == 'MANHATTAN'

In [25]:
path_noise ='hdfs:///user/xh895/BDM_Project/311_Noise_Complaints.csv'
path_comp_all = 'hdfs:////user/xh895/BDM_Project/311_dl_32817.csv'
path_local = '/Users/macbook/Google Drive/BDM-project/311_dl_32817.csv'
path_ct = '../data/nyct2010_16d/nyct2010.shp'
#noise  = sc.textFile(path_noise, 8)
#all_comp = sc.textFile(path_comp_all, 8)
all_comp = sc.textFile(path_local, 8)

In [27]:
res = all_comp.filter(ft_header).map(mp_col)
ress = res.filter(ft_noise).filter(ft_havetime).filter(ft_have_geo).map(mp_sjoin_ct).map(mp_groupbykey)
ress.take(200)

[((2014, u'3000100'), 1),
 ((2014, u'4011000'), 1),
 ((2014, u'1030300'), 1),
 ((2014, u'4025100'), 1),
 ((2014, u'3039300'), 1),
 ((2014, u'3034700'), 1),
 ((2014, u'1014802'), 1),
 ((2014, u'4084900'), 1),
 ((2014, u'2040600'), 1),
 ((2014, u'1012100'), 1),
 ((2014, u'1006800'), 1),
 ((2014, u'1022800'), 1),
 ((2014, u'3056500'), 1),
 ((2014, u'1005600'), 1),
 ((2014, u'1019500'), 1),
 ((2014, u'1012900'), 1),
 ((2014, u'3051400'), 1),
 ((2014, u'1019000'), 1),
 ((2014, u'2018500'), 1),
 ((2014, u'2040702'), 1),
 ((2014, u'3049300'), 1),
 ((2014, u'1019000'), 1),
 ((2014, u'1012200'), 1),
 ((2014, u'1003001'), 1),
 ((2014, u'2022500'), 1),
 ((2014, u'2019300'), 1),
 ((2014, u'1017900'), 1),
 ((2014, u'3073600'), 1),
 ((2014, u'3011600'), 1),
 ((2014, u'3037700'), 1),
 ((2014, u'2023502'), 1),
 ((2014, u'3014800'), 1),
 ((2014, u'1018800'), 1),
 ((2014, u'1008000'), 1),
 ((2014, u'2039100'), 1),
 ((2014, u'2036300'), 1),
 ((2014, u'3077000'), 1),
 ((2014, u'3035900'), 1),
 ((2014, u'1

In [28]:

print ress.reduceByKey(add).collect()

KeyboardInterrupt: 